Notebook prior to the INF8225 project to analyse which algorithm produces the best final clustering depending on multiple metrics (final cost, duration, number of iterations). Out of the scope of the INF8225 project.

Imports

In [464]:



import pandas as pd
import numpy as np
from pathlib import Path
from IPython.display import display, HTML
import plotly.express as px
import dataframe_image as dfi
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
pd.options.mode.chained_assignment = 'warn'  # default=
pd.options.plotting.backend = "plotly"
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('colheader_justify', 'center')
PATH_CACHE = Path("../../data/results_fi_bi.csv").resolve()
pd.set_option('display.max_columns', None)  

## Load original data and preprocess

In [465]:
orig_df = pd.read_csv(PATH_CACHE, sep=",",header=0,index_col=False)
# for col in orig_df:
#     print(col,orig_df[col].unique())
for orig_name,new_name in [(0,"BI"),(1,"FI"),(2,"DI")]:
    orig_df.loc[orig_df.ImprClass == orig_name,"ImprClass"] = new_name
for orig_name,new_name in [(0,"BACK"),(1,"CURR"),(2,"RANDOM")]:
    orig_df.loc[orig_df.iteration_order == orig_name,"iteration_order"] = new_name
for orig_name,new_name in [(0,"random"),(1,"kmeans+")]:
    orig_df.loc[orig_df.init_type == orig_name,"init_type"] = new_name
orig_df.loc[:,"algorithm"] = orig_df["ImprClass"]+"_" + orig_df["iteration_order"]
orig_df.loc[:,"Nb. points"] = orig_df["num_points"]
orig_df.drop(["ImprClass","iteration_order","num_points","seed"],axis=1,inplace=True)
df = orig_df.copy()
# print(display(orig_df.query("num_clusters==2 & init_type=='random' & algorithm=='FI_CURR'")))

### Temps observé

In [485]:
ALGO = "FI_CURR"
df_time = orig_df.copy()
val=np.sum(df_time.query(f"algorithm == '{ALGO}'")["time"])*1e-6 # en s
h = int(val/3600)
m = int((val-h*3600)/60)
s = int(val-h*3600-m*60)
print("Time:",h,"h",m,"m",s,"s")

Time: 4 h 44 m 41 s


## Merged Data

In [486]:
FIELDS = ["end_cost","time","num_iter","num_iter_tot"]

In [487]:
grouped_cols = ["num_clusters","Nb. points","algorithm","init_type"]
df_compil = orig_df.copy().groupby(grouped_cols).mean().reset_index()
Ldf = []
col_names = [("random","BI_BACK"),("random","FI_BACK"),("random","FI_CURR"),("random","% augm. FI_BACK"),("random","% augm. FI_CURR"),
             ("kmeans+","BI_BACK"),("kmeans+","FI_BACK"),("kmeans+","FI_CURR"),("kmeans+","% augm. FI_BACK"),("kmeans+","% augm. FI_CURR")
             ]
for field in FIELDS:
    df = df_compil.set_index(grouped_cols)[field].unstack().unstack()
    for init in ["random","kmeans+"]:
        for fi in ["FI_BACK","FI_CURR"]:
            df[init,"% augm. "+fi] = (df[init,fi]-df[init,"BI_BACK"])/df[init,"BI_BACK"]*100
    df = df[col_names]
    df = pd.concat([df], axis=1, keys=[field]) # Add field name to columns
    Ldf.append(df)
df_compil = pd.concat(Ldf, axis=1)
df_compil = df_compil.reset_index().drop(["Nb. points"],axis=1).set_index("num_clusters")
# display(df_compil)
for f in FIELDS:
    display(df_compil[[f]])

C:\Users\robin\AppData\Local\Temp\ipykernel_1228\1945538580.py:16: PerformanceWarning:

dropping on a non-lexsorted multi-index without a level parameter may impact performance.



end_cost                                              \
init_type       random                                                
algorithm      BI_BACK      FI_BACK      FI_CURR    % augm. FI_BACK   
num_clusters                                                          
2            1,030,079.28 1,052,649.77 1,049,107.92         2.19      
4              413,903.17   541,583.69   544,726.83        30.85      
8              211,770.48   541,871.99   575,471.25       155.88      
16             102,322.38   541,905.44   575,264.81       429.61      
32              49,055.96   608,219.31   666,539.49     1,139.85      
64              23,515.97   634,924.87   687,167.75     2,599.97      
128             10,520.54   590,615.33   619,982.11     5,513.92      
256              3,703.82   476,977.44   511,158.74    12,778.00      

                                                                     \
init_type                      kmeans+                                
algorithm    % augm. FI_CURR   BI_BACK      FI_BACK      FI_CURR      
num_clusters                                                          
2                    1.85    1,583,164.60 1,586,455.92 1,582,858.71   
4                   31.61    1,443,572.15 1,442,080.41 1,447,817.54   
8                  171.74    1,530,993.81 1,535,985.34 1,530,183.21   
16                 462.21    1,577,687.04 1,579,973.49 1,579,800.29   
32               1,258.73    1,584,017.72 1,583,804.08 1,585,308.47   
64               2,822.13    1,544,217.57 1,547,452.93 1,546,382.81   
128              5,793.06    1,446,970.07 1,446,704.29 1,446,322.76   
256             13,700.86    1,246,042.30 1,246,529.83 1,245,851.79   

                                              
init_type                                     
algorithm    % augm. FI_BACK % augm. FI_CURR  
num_clusters                                  
2                  0.21           -0.02       
4                 -0.10            0.29       
8                  0.33           -0.05       
16                 0.14            0.13       
32                -0.01            0.08       
64                 0.21            0.14       
128               -0.02           -0.04       
256                0.04           -0.02

time                                                \
init_type       random                                                
algorithm      BI_BACK      FI_BACK      FI_CURR    % augm. FI_BACK   
num_clusters                                                          
2                9,475.51     6,036.15     5,805.13      -36.30       
4               40,993.40    29,707.16    37,861.39      -27.53       
8              119,703.27   132,509.73   141,422.74       10.70       
16             294,998.26   444,285.95   470,387.48       50.61       
32             651,564.32 1,004,434.93 1,072,246.39       54.16       
64           1,267,183.00 1,640,063.00 2,113,844.14       29.43       
128          2,399,886.46 3,758,853.47 4,347,150.77       56.63       
256          3,894,641.72 5,976,909.46 7,032,811.70       53.46       

                                                                 \
init_type                      kmeans+                            
algorithm    % augm. FI_CURR   BI_BACK     FI_BACK    FI_CURR     
num_clusters                                                      
2                 -38.74         3,725.17   2,825.93   2,270.99   
4                  -7.64        21,974.78  14,368.35  12,617.00   
8                  18.14        53,300.16  42,333.36  39,850.10   
16                 59.45       116,383.87  92,560.44  98,915.74   
32                 64.56       217,642.91 172,427.85 164,885.93   
64                 66.81       392,115.09 311,681.05 281,201.57   
128                81.14       693,547.96 579,400.57 467,813.48   
256                80.58     1,025,515.51 967,973.99 792,030.53   

                                              
init_type                                     
algorithm    % augm. FI_BACK % augm. FI_CURR  
num_clusters                                  
2                 -24.14          -39.04      
4                 -34.61          -42.58      
8                 -20.58          -25.23      
16                -20.47          -15.01      
32                -20.77          -24.24      
64                -20.51          -28.29      
128               -16.46          -32.55      
256                -5.61          -22.77

num_iter                                                    \
init_type     random                                                      
algorithm    BI_BACK  FI_BACK  FI_CURR  % augm. FI_BACK % augm. FI_CURR   
num_clusters                                                              
2              510.15   817.11   819.38       60.17           60.61       
4              886.30 2,062.68 2,053.62      132.73          131.71       
8            1,217.48 3,587.72 3,569.65      194.68          193.20       
16           1,399.40 5,556.27 5,654.24      297.05          304.05       
32           1,534.25 7,221.29 7,129.06      370.67          364.66       
64           1,571.68 8,588.02 8,440.80      446.42          437.05       
128          1,491.85 9,873.52 9,919.03      561.83          564.88       
256          1,284.59 9,585.79 9,234.18      646.22          618.85       

                                                                      
init_type    kmeans+                                                  
algorithm    BI_BACK FI_BACK FI_CURR % augm. FI_BACK % augm. FI_CURR  
num_clusters                                                          
2             175.07  185.94  190.63       6.21            8.89       
4             395.61  512.55  512.11      29.56           29.45       
8             504.69  754.40  763.66      49.48           51.31       
16            537.01  826.93  828.81      53.99           54.34       
32            504.39  784.41  796.97      55.52           58.00       
64            451.08  708.27  711.75      57.02           57.79       
128           397.19  627.29  639.87      57.93           61.10       
256           345.61  529.58  524.62      53.23           51.79

num_iter_tot                                                 \
init_type        random                                                     
algorithm       BI_BACK        FI_BACK        FI_CURR     % augm. FI_BACK   
num_clusters                                                                
2                511,154.00     330,086.96     330,109.53      -35.42       
4              2,661,888.00   2,328,509.02   2,315,292.78      -12.52       
8              8,529,388.00   9,013,647.59   8,907,540.10        5.68       
16            21,006,000.00  28,892,365.27  28,773,720.54       37.54       
32            47,592,874.00  70,619,655.86  68,060,844.21       48.38       
64            99,079,092.00 150,153,325.33 142,517,872.09       51.55       
128          189,591,696.00 297,514,238.76 297,086,015.95       56.92       
256          327,824,175.00 501,769,959.20 475,185,237.97       53.06       

                                                                        \
init_type                       kmeans+                                  
algorithm    % augm. FI_CURR    BI_BACK       FI_BACK       FI_CURR      
num_clusters                                                             
2                 -35.42        176,066.00     90,808.66     92,835.18   
4                 -13.02      1,189,836.00    722,785.83    719,630.33   
8                   4.43      3,539,858.00  2,411,948.65  2,445,188.58   
16                 36.98      8,070,090.00  5,655,091.60  5,652,694.24   
32                 43.01     15,667,245.00 11,028,412.75 11,202,833.55   
64                 43.84     28,480,977.00 19,965,781.36 20,118,279.23   
128                56.70     50,569,749.00 35,151,860.10 36,060,293.02   
256                44.95     88,385,805.00 58,870,454.49 58,396,002.09   

                                              
init_type                                     
algorithm    % augm. FI_BACK % augm. FI_CURR  
num_clusters                                  
2                 -48.42          -47.27      
4                 -39.25          -39.52      
8                 -31.86          -30.92      
16                -29.93          -29.96      
32                -29.61          -28.50      
64                -29.90          -29.36      
128               -30.49          -28.69      
256               -33.39          -33.93

### Temps avec moyennes

In [484]:
ALGO = "FI_BACK"
df_time = Ldf[1].copy()
tot = 0.
for init in ["random","kmeans+"]:
    tot += np.sum(df_time["time",init,ALGO])
val=tot*1e-6*1000 # en s
h = int(val/3600)
m = int((val-h*3600)/60)
s = int(val-h*3600-m*60)
print("Time:",h,"h",m,"m",s,"s")


Time: 4 h 12 m 56 s


In [470]:
ALGO = "FI_CURR"
for FIELD,(name,ticks_pow,sign) in {"end_cost":("coût final",[-2,-1,0],True),"time":("temps d'exécution",[0,1,2],True),"num_iter":("Nb. iterations",[0,1,2,3],False),"num_iter_tot":("Nb. iterations tot.",[0,1,2],True)}.items():
    df = df_compil[FIELD]
    Ldf = []
    for init in ["random","kmeans+"]:
        df_init = pd.concat([df[(init,"% augm. "+ALGO)]],axis=1,keys=["% augm. "+ALGO]).reset_index()
        df_init["init_type"] = init
        Ldf.append(df_init)
    df = pd.concat(Ldf,axis=0)

    max_val = max(abs(x) for x in ticks_pow)
    min_ticks = min(ticks_pow)
    def logscale(x):
        x = np.array(x)
        log_val = np.log10(np.abs(x+1e-16))
        clipped = np.clip(log_val,min_ticks-1,np.inf)
        return np.sign(x) * (1-min_ticks+clipped)
    colors = ["green","#00FF00","white","orange","red"] if sign else ["white","orange","red"]
    fig = px.scatter(df,x=np.log2(df["num_clusters"]).astype(int),y="init_type",color=logscale(df["% augm. "+ALGO]),# x="Nb. points",
                        hover_data=["num_clusters","init_type","% augm. "+ALGO],
                        color_continuous_scale=colors,
                        range_color=[-max_val-delta_zero if sign else 0.,max_val+delta_zero],
                        range_x=[0,9],
                        width=800,height=400,
                        title=f"% augmentation du {name}<br>pour 1000 points de {ALGO} par rapport à BI_BACK"
                        )

    def build(f):
        ticks_pos = [f(1,p) for p in ticks_pow]
        if sign:
            ticks_neg = [f(-1,p) for p in ticks_pow[::-1]]
        else:
            ticks_neg = []
        return [*ticks_neg,logscale(0),*ticks_pos]
    def func(sign,p):
        real_value = sign*10**p
        return logscale(real_value)
    Lp = build(func)
    Lannot = build(lambda sign,p: "1E"+str(p) if sign > 0 else "-1E"+str(p))
    # print(Lp)
    # print(Lannot)
    fig.update_layout(
        xaxis_title="Nombre de clusters",
        yaxis_title="Initialisation",
        coloraxis_colorbar=dict(
        title=f"% augmentation <br>{name}",
        tickvals=np.round(Lp,decimals=2),
        ticktext=Lannot,
    ))
    fig.update_xaxes(
        ticktext=[str(2**i) for i in range(9)],
        tickvals=[i for i in range(9)],
    )
    fig.update_traces(marker=dict(size=12,
                                line=dict(width=2,
                                            color='black')),
                    selector=dict(mode='markers'))
    fig.show()